## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [2]:
import os
import torch
import chassisml
import numpy as np

## Enter credentials
Dockerhub creds and Modzy API Key

In [4]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


## Prepare model, labels, and data transform object
Initialize anything here that should persist across inference runs

In this example, we will download this [DistilBERT](https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion?text=I+like+you.+I+love+you) pretrained model. Modify the below import based on the "Use in Transformers" code for the model you wish to containerize.

In [3]:
# Download model with the transformers library
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

In [5]:
# First, we save model locally so we can load it into memory for use with Chassis package
tokenizer.save_pretrained("./model")
model.save_pretrained("./model")

In [6]:
# Load model into memory from local file
distilbert_tokenizer = AutoTokenizer.from_pretrained("./model")
distilbert_model = AutoModelForSequenceClassification.from_pretrained("./model")

# define labels from model config
labels = model.config.id2label

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [7]:
# define process function that will serve as our inference function
def process(input_bytes):
    # decode and preprocess data bytes
    text = input_bytes.decode()
    inputs = distilbert_tokenizer(text, return_tensors="pt")
    
    # run preprocessed data through model
    with torch.no_grad():
        logits = distilbert_model(**inputs).logits
        softmax = torch.nn.functional.softmax(logits, dim=1).detach().cpu().numpy()
        
    # postprocess 
    indices = np.argsort(softmax)[0][::-1]
    results = {
        "data": {
            "result": {
                "classPredictions": [{"class": labels[i], "score": softmax[0][i]} for i in indices]
            }
        }
    }
    
    return results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [8]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `process_fn` defined above

In [9]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/input.txt'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"classPredictions":[{"class":"joy","score":0.9971561431884766},{"class":"love","score":0.0008850579615682364},{"class":"sadness","score":0.0008116100216284394},{"class":"anger","score":0.0005449091549962759},{"class":"surprise","score":0.00035346485674381256},{"class":"fear","score":0.00024872180074453354}]}}}'


In [10]:
# test environment and model within Chassis service, must pass filepath here:
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [11]:
response = chassis_model.publish(
    model_name="Hugging Face DistilBERT",
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass
)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!
